ref: https://youtu.be/H9OmsD7F7p0

In [ ]:
!pip install pythainlp
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = '/content/drive/MyDrive/Final_project/clean_data/combine_company.csv'

In [4]:
import pandas as pd
import numpy as np
from pythainlp.corpus import thai_stopwords
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
df = pd.read_csv(data)
frozen_stopword = thai_stopwords()

In [23]:
# Clean stop word and special charater
import string
from pythainlp.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopword = stopwords.words('english')
thai_stopword = list(frozen_stopword)

def clean_tokenize(rec_text:str) -> list:
  clean_token = rec_text
  for pair in (('\n', ''), ('\t', ''), ('!', '')):
        clean_token =clean_token.replace(*pair)
  
  temp_tokenize = word_tokenize(clean_token, None, 'newmm', False)
  return temp_tokenize.lower()

def clean_stopword(word:list):
  temp_thai = []
  temp_eng = []
  for i in word:
    if i not in thai_stopword:
      temp_thai.append(i)
  
  for j in temp_thai:
    if j not in english_stopword:
      temp_eng.append(j)
    
  return temp_eng

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# List company detail
"""
  ตัดรายละเอียดธุรกิจเป็นคำๆ แล้วใส่ในตัวแปร list_company_detail
"""

list_company_detail = []
for i in range(len(df)):
  a = df.iloc[i]['รายละเอียดธุรกิจ'].translate(str.maketrans('','', string.punctuation))
  b = a.translate(str.maketrans('','', string.digits))
  temp_comp = clean_tokenize(b)
  stopword_cleaned = clean_stopword(temp_comp)
  list_company_detail.append(stopword_cleaned)

# หาค่า TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
def identity_fun(text):
    return text

In [25]:
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.5,
    max_features=6000,
    min_df=2,
    use_idf=True,
    stop_words='english',
    analyzer='word',
    tokenizer=identity_fun,
    preprocessor=identity_fun,
    token_pattern=None
)

X = tfidf_vectorizer.fit_transform(list_company_detail)
# tfidf_array = np.array(tfidf_vector.todense())

#แปลงเป็น DataFrame เพื่อง่ายแก่การอ่าน
# df1 = pd.DataFrame(tfidf_array,columns=tfidf_vectorizer.get_feature_names_out())
# df1.shape

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn(


# KMeans clustering

In [26]:
from sklearn.cluster import KMeans

In [27]:
km = KMeans(n_clusters=8)

In [28]:
km.fit(X)

KMeans()

In [29]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

In [47]:
centroid = km.cluster_centers_
cluster = km.labels_

In [44]:
df['cluster'] = cluster

In [45]:

cluster_map = {
    0: "e-commerce",
    1: "Chatbot",
    # 1: "Robotics",
    2: "Software Development",
    3: "Artificial Intelligence",
    4: "Mobile",
    5: "Internet of Things",
    6: "Biometrics",
    7: "Data Science"
}
# Embedded System

# apply mapping
df['cluster'] = df['cluster'].map(cluster_map)

In [54]:
import random
for i in range(5):
  ind_sample = random.randint(0, len(df))
  print(f'{df.iloc[ind_sample]["ชื่อย่อสถานประกอบการ"]}')
  print(df.iloc[ind_sample]['รายละเอียดธุรกิจ'])
  print(f'cluster label: {cluster[ind_sample]}')
  print(f"Category: {df.iloc[ind_sample]['cluster']}")
  print()

BONZEN
บริการออกแบบพัฒนาจัดหาระบบเทคโนโลยีสารสนเทศ IT Solution, Software Development Online Platform Web/Mobile Application, Building and Facility management, Energy Data Management, Engineering data management, construction data management,
cluster label: 2
Category: Software Development

Feedback180
บริการงานออกแบบและพัฒนาซอฟต์แวร์อัจฉริยะ (AI Technology) สำหรับวิเคราะห์ข้อมูลทั้งในรูปแบบถ้อยคำและภาพเคลื่อนไหว, บริการจัดหา Solution - Customer VoC360 Insights / Customer Closed Loop Feedback /VDO Analytics Shopper Experience โดยใช้เทคโนโลยี NLP (Natural Language Processing) บวกกับ Machine Learning และ Deep Learning ร่วมกับความรู้ทางด้านภาษาศาสตร์ของทีมงานนักภาษาศาสตร์ (Linguist) และ Data Science ในการวิเคราะห์ “ภาษาธรรมชาติ” หรือภาษาที่เราใช้จริงในชีวิตประจำวัน (ไม่ใช่แค่วิเคราะห์ภาษาตามหลักภาษาไทยในตำราเรียนทั่วไป) ซึ่งเราเก็บรวบรวมเสียงสะท้อนของลูกค้าจากทุกช่องทางทั้ง Online และ Offline แล้วมาสร้างโมเดลที่ช่วยค้นหา Customer Experience,
cluster label: 0
Category: e-commerce

aris
บริก